# Practical 4: Concurrency Control (Part 1)

**Course**: BMCS3003 Distributed Systems and Parallel Computing

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 90 minutes

**Prerequisites**:
- Practical 3: OpenMP basics
- Understanding of shared vs private data
- Parallel loop concepts

## Learning Objectives

By the end of this practical, you will be able to:

1. Understand and prevent race conditions using critical sections
2. Implement parallel reduction operations efficiently
3. Apply the producer-consumer pattern with OpenMP
4. Use sections directive for task parallelism
5. Implement atomic operations for lock-free synchronization
6. Understand flush directive for memory consistency

## Table of Contents

1. [Critical Sections and Race Conditions](#section1)
2. [Question 1: Pi Calculation with Critical Section](#section2)
3. [Question 2: Array Average with Reduction](#section3)
4. [Producer-Consumer Pattern](#section4)
5. [Question 3: Producer-Consumer Implementation](#section5)
6. [Atomic Operations](#section6)
7. [Question 4: Avoiding Race Conditions](#section7)
8. [Summary](#section8)

<a id='section1'></a>
## 1. Critical Sections and Race Conditions

### What is a Race Condition?

A **race condition** occurs when:
- Multiple threads access shared data
- At least one thread modifies the data
- No synchronization between threads
- The result depends on thread execution order

### Example: Race Condition

```cpp
int counter = 0;  // Shared variable

#pragma omp parallel for
for (int i = 0; i < 1000; i++) {
    counter++;  // RACE CONDITION!
}

printf("Counter: %d\n", counter);  // Expected: 1000, Actual: ???
```

#### Why is this wrong?

The operation `counter++` is NOT atomic. It consists of three steps:

```
1. READ:  temp = counter
2. ADD:   temp = temp + 1
3. WRITE: counter = temp
```

#### Thread Interleaving Example

```
Initial: counter = 5

Thread 0                    Thread 1
READ counter (5)            
ADD: temp = 6               
                            READ counter (5)  ← Still 5!
WRITE counter = 6           
                            ADD: temp = 6
                            WRITE counter = 6  ← Lost update!

Result: counter = 6 (should be 7)
```

### Solution: Critical Section

A **critical section** ensures only ONE thread executes the protected code at a time.

```cpp
int counter = 0;

#pragma omp parallel for
for (int i = 0; i < 1000; i++) {
    #pragma omp critical
    {
        counter++;  // Protected!
    }
}

printf("Counter: %d\n", counter);  // Always 1000
```

### How Critical Section Works

```
Thread 0         Thread 1         Thread 2
   |                |                |
   | Enter          |                |
   | critical       | Wait...        | Wait...
   | section        |                |
   | counter++      |                |
   | Exit           |                |
   |                | Enter          |
   |                | critical       | Wait...
   |                | counter++      |
   |                | Exit           |
   |                |                | Enter
   |                |                | critical
   |                |                | counter++
   |                |                | Exit
```

### Named Critical Sections

You can have multiple independent critical sections:

```cpp
int sum_x = 0, sum_y = 0;

#pragma omp parallel
{
    #pragma omp critical(update_x)
    sum_x++;
    
    #pragma omp critical(update_y)
    sum_y++;
}
```

- Unnamed critical sections: All are mutually exclusive
- Named critical sections: Only same name are mutually exclusive

### False Sharing Recap

From Practical 3, we learned about **false sharing**:

```cpp
// BAD: False sharing
double sum[8];  // All in same cache line

#pragma omp parallel
{
    int id = omp_get_thread_num();
    sum[id] = 0;  // Cache line invalidation!
}
```

**Problems with Critical Section + False Sharing**:
1. Critical section prevents race conditions
2. BUT false sharing still causes cache invalidation
3. Performance loss from both serialization AND cache misses

**Solution**: Use reduction clause (covered in Question 1)

<a id='section2'></a>
## Question 1: Pi Calculation with Critical Section

### Objective

Modify the Pi program from Practical 3 to avoid false sharing using:
1. Scalar local variable (no array)
2. Critical section to protect final sum

### Why This Approach?

**Previous approach (Practical 3)**:
```cpp
double sum[16];  // Array → false sharing
#pragma omp parallel
{
    int id = omp_get_thread_num();
    sum[id] = partial_calculation();
}
```

**New approach**:
```cpp
double pi = 0.0;
#pragma omp parallel
{
    double local_sum = partial_calculation();  // No sharing!
    
    #pragma omp critical
    {
        pi += local_sum;  // Protected update
    }
}
```

### Implementation (P4Q1.cpp)

```cpp
#include <iostream>
#include <omp.h>

static long num_steps = 10000000;
double step;

int main() {
    double pi = 0.0;  // Final result (shared)
    step = 1.0 / (double)num_steps;
    
    double start_time = omp_get_wtime();
    
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        int num_threads = omp_get_num_threads();
        
        // Step 1: Create scalar local variable
        // Each thread has its OWN sum (no sharing, no false sharing)
        double local_sum = 0.0;
        
        // Step 2: Each thread computes its portion
        for (int i = id; i < num_steps; i += num_threads) {
            double x = (i + 0.5) * step;
            local_sum += 4.0 / (1.0 + x * x);
        }
        
        // Step 3: Protect summation into pi with critical section
        // local_sum goes "out of scope" after parallel region,
        // so we MUST sum it here
        #pragma omp critical
        {
            pi += local_sum;
        }
    }
    
    pi *= step;  // Multiply by width
    
    double end_time = omp_get_wtime();
    
    printf("Pi = %.10f\n", pi);
    printf("Time = %.6f seconds\n", end_time - start_time);
    
    return 0;
}
```

### Expected Output

```
Pi = 3.1415926536
Time = 0.002832 seconds
```

### Analysis

**Benefits**:
1. No array → No false sharing
2. Local sum → Each thread works independently
3. Critical section → Only used ONCE per thread (not per iteration)

**Critical section cost**:
- Only 8 critical section entries (for 8 threads)
- Much better than 10,000,000 entries if we protected `pi +=` inside loop!

### Understanding Variable Scope

```cpp
double pi = 0.0;  // Declared BEFORE parallel → SHARED

#pragma omp parallel
{
    double local_sum = 0.0;  // Declared INSIDE parallel → PRIVATE
    
    // local_sum is different for each thread
    // pi is same for all threads
    
    #pragma omp critical
    {
        pi += local_sum;  // Safe: protected by critical
    }
    
} // local_sum destroyed here (out of scope)

// pi still exists and has final result
```

### Clauses Reference

| Clause | Description | Example |
|--------|-------------|--------|
| `shared(var)` | Explicitly shared | `shared(pi)` |
| `private(var)` | Thread-private copy | `private(local_sum)` |
| `critical` | Mutual exclusion | `#pragma omp critical` |
| `critical(name)` | Named critical section | `#pragma omp critical(update_pi)` |

<a id='section3'></a>
## Question 2: Array Average Computation

### Problem Statement

Compute the average of an array with 100,000 elements in parallel.

**Serial code**:
```cpp
double ave = 0.0, A[MAX];
int i;

for (i = 0; i < MAX; i++) {
    ave += A[i];
}
ave = ave / MAX;
```

### Method 1: Using `#pragma omp parallel` and `#pragma omp critical`

```cpp
#include <iostream>
#include <omp.h>
#include <cstdlib>
#include <ctime>

#define MAX 100000

double A[MAX];

void initialize_array() {
    srand(time(NULL));
    for (int i = 0; i < MAX; i++) {
        A[i] = rand() % 100;  // Random 0-99
    }
}

int main() {
    initialize_array();
    
    double ave = 0.0;
    
    double start = omp_get_wtime();
    
    #pragma omp parallel
    {
        int id = omp_get_thread_num();
        int num_threads = omp_get_num_threads();
        
        // Local sum for this thread
        double local_sum = 0.0;
        
        // Divide work among threads
        for (int i = id; i < MAX; i += num_threads) {
            local_sum += A[i];
        }
        
        // Add local sum to global average (protected)
        #pragma omp critical
        {
            ave += local_sum;
        }
    }
    
    ave = ave / MAX;
    
    double end = omp_get_wtime();
    
    printf("Average = %.2f\n", ave);
    printf("Time = %.6f seconds\n", end - start);
    
    return 0;
}
```

### Method 2: Using `#pragma omp parallel for reduction(+:ave)`

**Much simpler and more efficient!**

```cpp
#include <iostream>
#include <omp.h>
#include <cstdlib>
#include <ctime>

#define MAX 100000

double A[MAX];

void initialize_array() {
    srand(time(NULL));
    for (int i = 0; i < MAX; i++) {
        A[i] = rand() % 100;
    }
}

int main() {
    initialize_array();
    
    double ave = 0.0;
    
    double start = omp_get_wtime();
    
    // One line of code!
    #pragma omp parallel for reduction(+:ave)
    for (int i = 0; i < MAX; i++) {
        ave += A[i];
    }
    
    ave = ave / MAX;
    
    double end = omp_get_wtime();
    
    printf("Average = %.2f\n", ave);
    printf("Time = %.6f seconds\n", end - start);
    
    return 0;
}
```

### Performance Comparison

| Implementation | Time (ms) | Speedup | Notes |
|----------------|-----------|---------|-------|
| Sequential | 0.452 | 1.0x | Baseline |
| Parallel + Critical | 0.421 | 1.07x | Minimal speedup |
| Parallel + Reduction | 0.089 | 5.08x | Best performance |

### Discussion: Why is Sequential Fast?

**Question**: Why does sequential run at similar speed (or even faster) than parallel with critical section?

**Answer**:

1. **Task is too simple**: Adding array elements is very fast (few CPU cycles)
2. **Memory bound**: Limited by memory bandwidth, not computation
3. **Overhead dominates**: 
   - Thread creation: ~microseconds
   - Critical section contention: Threads wait for each other
   - Context switching: OS switches between threads

4. **Critical section serialization**:
   ```
   Thread 0: Compute → Wait → Update → Done
   Thread 1: Compute → Wait → Update → Done
   Thread 2: Compute → Wait → Update → Done
                        ↑
                   Bottleneck!
   ```

5. **Reduction is optimized**: OpenMP uses tree-based reduction (logarithmic cost)

<a id='section4'></a>
## Producer-Consumer Pattern

### What is Producer-Consumer?

A common parallel pattern where:
- **Producer**: Generates data
- **Consumer**: Processes data
- **Buffer**: Stores data between producer and consumer

### Use Cases

- Data processing pipelines
- Network servers (receive → process)
- Video streaming (decode → display)
- Log processing (collect → analyze)

### Basic Pattern

```
Producer Thread          Consumer Thread
     |                        |
     | Generate data          |
     | Write to buffer        |
     |                        | Read from buffer
     |                        | Process data
     | Generate more...       |
     |                        | Process more...
```

### Synchronization Challenges

1. **Data ready signal**: Consumer must wait for data
2. **Memory consistency**: Ensure data is visible to consumer
3. **Race conditions**: Protect shared buffer

### OpenMP Directives for Producer-Consumer

#### 1. `#pragma omp sections`

Divides work into independent sections:

```cpp
#pragma omp parallel sections
{
    #pragma omp section
    {
        // Section 1: Executed by one thread
    }
    
    #pragma omp section
    {
        // Section 2: Executed by another thread
    }
}
```

#### 2. `#pragma omp flush`

Ensures memory consistency:

```cpp
int data = 0;
bool data_ready = false;

// Producer
data = 42;
#pragma omp flush(data)  // Make sure data is written to memory
data_ready = true;
#pragma omp flush(data_ready)  // Make sure flag is visible

// Consumer
while (!data_ready) {
    #pragma omp flush(data_ready)  // Read latest value
}
#pragma omp flush(data)  // Read latest data
process(data);
```

<a id='section5'></a>
## Question 3: Producer-Consumer Implementation

### Problem Statement

Implement a producer-consumer pattern where:
- Producer generates numbers and sums them
- Consumer receives the final sum
- Use `#pragma omp sections` and `#pragma omp flush`

### Implementation (P4Q3.cpp)

```cpp
#include <iostream>
#include <omp.h>

#define N 1000000

int main() {
    double sum = 0.0;
    bool data_ready = false;  // Flag: is data ready?
    
    double start_time = omp_get_wtime();
    
    #pragma omp parallel sections
    {
        // Producer section
        #pragma omp section
        {
            printf("Producer: Starting calculation...\n");
            
            // Produce data: calculate sum
            for (int i = 0; i < N; i++) {
                sum += i;
            }
            
            printf("Producer: Sum calculated = %.0f\n", sum);
            
            // Signal data is ready
            #pragma omp flush(sum)  // Ensure sum is written to memory
            data_ready = true;
            #pragma omp flush(data_ready)  // Ensure flag is visible
            
            printf("Producer: Data ready signal sent\n");
        }
        
        // Consumer section
        #pragma omp section
        {
            printf("Consumer: Waiting for data...\n");
            
            // Wait for data to be ready
            while (!data_ready) {
                #pragma omp flush(data_ready)  // Check latest flag value
            }
            
            // Read the data
            #pragma omp flush(sum)  // Read latest sum value
            
            printf("Consumer: Received sum = %.0f\n", sum);
            
            // Process the data (verify it's correct)
            double expected = (double)N * (N - 1) / 2.0;  // Formula: n*(n-1)/2
            
            if (sum == expected) {
                printf("Consumer: Verification PASSED\n");
            } else {
                printf("Consumer: Verification FAILED (expected %.0f)\n", expected);
            }
        }
    }
    // Implicit barrier here - both sections must complete
    
    double end_time = omp_get_wtime();
    
    printf("\nTotal time: %.6f seconds\n", end_time - start_time);
    
    return 0;
}
```

### Expected Output

```
Producer: Starting calculation...
Consumer: Waiting for data...
Producer: Sum calculated = 499999500000
Producer: Data ready signal sent
Consumer: Received sum = 499999500000
Consumer: Verification PASSED

Total time: 0.006255 seconds
```

### Understanding the Flow

```
Time →

Producer:  [Calculate sum..................] Signal ready
Consumer:  [Wait....................] Read & process
           ↑                        ↑
       Both start                Data ready
```

### Key Concepts

#### 1. Sections Directive
```cpp
#pragma omp parallel sections
```
- Creates parallel region with independent sections
- Each section executed by different thread
- Implicit barrier at end (all sections must complete)

#### 2. Flush Directive
```cpp
#pragma omp flush(variable)
```
- Ensures memory consistency
- Makes variable visible to all threads
- Required for communication between threads

#### 3. Busy Waiting
```cpp
while (!data_ready) {
    #pragma omp flush(data_ready);
}
```
- Consumer actively checks flag
- Burns CPU cycles (not ideal for long waits)
- Acceptable for short waits in compute-bound programs

### Pipeline Parallelism Extension

**Advanced**: Multiple producer-consumer stages

```cpp
#pragma omp parallel sections
{
    #pragma omp section
    {
        // Stage 1: Generate data
        produce_data();
    }
    
    #pragma omp section
    {
        // Stage 2: Transform data
        transform_data();
    }
    
    #pragma omp section
    {
        // Stage 3: Process results
        process_results();
    }
}
```

### Common Issues

#### Issue 1: Missing flush
```cpp
// Producer
data = 42;
data_ready = true;  // Missing flush!

// Consumer may never see data_ready = true
```

#### Issue 2: Wrong flush order
```cpp
// Producer
data_ready = true;
#pragma omp flush(data_ready)
data = 42;  // TOO LATE!
#pragma omp flush(data)

// Consumer may see flag=true but old data
```

**Correct order**:
1. Write data
2. Flush data
3. Write flag
4. Flush flag

<a id='section6'></a>
## Atomic Operations

### What are Atomic Operations?

**Atomic** = "one at a time" - operation appears to execute instantaneously

An atomic operation:
- Cannot be interrupted
- Completes entirely or not at all
- No partial updates visible to other threads

### Atomic vs Critical Section

```cpp
// Critical section (general purpose)
#pragma omp critical
{
    x[idx]++;  // Can be any statement
    y[idx] *= 2;  // Multiple statements
}

// Atomic (specific operations only)
#pragma omp atomic
x[idx]++;  // Only simple updates
```

| Feature | Critical | Atomic |
|---------|----------|--------|
| Scope | Multiple statements | Single statement |
| Operations | Any | Update, read, write, capture |
| Performance | Slower | Faster |
| Hardware support | No | Yes (CPU instructions) |

### Supported Atomic Operations

```cpp
// Atomic update (default)
#pragma omp atomic
x++;           // x = x + 1
x += expr;     // x = x + expr
x *= expr;     // x = x * expr
x -= expr;     // x = x - expr
x /= expr;     // x = x / expr
x &= expr;     // x = x & expr
x |= expr;     // x = x | expr
x ^= expr;     // x = x ^ expr

// Atomic write
#pragma omp atomic write
x = expr;

// Atomic read
#pragma omp atomic read
v = x;

// Atomic capture (read + update)
#pragma omp atomic capture
{
    v = x;
    x++;
}
```

### Example: Avoiding Race Conditions

```cpp
int x[100], y[100];

#pragma omp parallel for
for (int i = 0; i < 100; i++) {
    int idx = some_function(i);
    
    // Problem: Multiple iterations may have same idx
    // Race condition if idx values collide!
    
    #pragma omp atomic
    x[idx]++;  // Protected from race condition
    
    // NOTE: y[idx] is NOT protected!
    y[idx]++;  // Still has race condition
}
```

**Important**: Atomic directive applies ONLY to the immediately following statement!

### Advantages of Atomic

1. **Lock-free**: Uses CPU atomic instructions (e.g., `lock xadd` on x86)
2. **Fine-grained**: Each array element can be updated independently
3. **Better performance**: No lock contention

```cpp
// Critical section: x[0] and x[1] CANNOT be updated simultaneously
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp critical
    x[i]++;  // Serialized!
}

// Atomic: x[0] and x[1] CAN be updated simultaneously
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp atomic
    x[i]++;  // Parallel!
}
```

<a id='section7'></a>
## Question 4: Array Updates with Atomic

### Problem Statement

Given code that updates array elements, add atomic constructs to ensure correctness.

### Original Code (P4Q4.cpp - with race conditions)

```cpp
#include <iostream>
#include <omp.h>

#define N 10000

int main() {
    int x[100] = {0}, y[100] = {0};
    
    #pragma omp parallel for
    for (int i = 0; i < N; i++) {
        int idx_x = i % 100;  // Index for x (0-99)
        int idx_y = i % 50;   // Index for y (0-49)
        
        // Race condition: Multiple threads may access same index
        x[idx_x]++;
        y[idx_y]++;
    }
    
    // Verify results
    long sum_x = 0, sum_y = 0;
    for (int i = 0; i < 100; i++) sum_x += x[i];
    for (int i = 0; i < 50; i++) sum_y += y[i];
    
    printf("Sum of x: %ld (expected: %d)\n", sum_x, N);
    printf("Sum of y: %ld (expected: %d)\n", sum_y, N);
    
    if (sum_x == N && sum_y == N) {
        printf("✓ Results correct!\n");
    } else {
        printf("✗ Race condition detected!\n");
    }
    
    return 0;
}
```

### Output (With Race Conditions)

```
Sum of x: 9847 (expected: 10000)
Sum of y: 9923 (expected: 10000)
✗ Race condition detected!
```

**Problem**: Lost updates when threads access same index simultaneously

### Solution: Add Atomic Constructs

```cpp
#include <iostream>
#include <omp.h>

#define N 10000

int main() {
    int x[100] = {0}, y[100] = {0};
    
    double start = omp_get_wtime();
    
    #pragma omp parallel for
    for (int i = 0; i < N; i++) {
        int idx_x = i % 100;
        int idx_y = i % 50;
        
        // Protect x[idx_x] update
        #pragma omp atomic
        x[idx_x]++;
        
        // Protect y[idx_y] update
        #pragma omp atomic
        y[idx_y]++;
    }
    
    double end = omp_get_wtime();
    
    // Verify results
    long sum_x = 0, sum_y = 0;
    for (int i = 0; i < 100; i++) sum_x += x[i];
    for (int i = 0; i < 50; i++) sum_y += y[i];
    
    printf("Sum of x: %ld (expected: %d)\n", sum_x, N);
    printf("Sum of y: %ld (expected: %d)\n", sum_y, N);
    
    if (sum_x == N && sum_y == N) {
        printf("✓ Results correct!\n");
    } else {
        printf("✗ Race condition detected!\n");
    }
    
    printf("Time: %.6f seconds\n", end - start);
    
    return 0;
}
```

### Output (Corrected)

```
Sum of x: 10000 (expected: 10000)
Sum of y: 10000 (expected: 10000)
✓ Results correct!
Time: 0.001245 seconds
```

### Why Atomic is Better Than Critical Here

#### Using Critical (slower)
```cpp
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp critical
    {
        x[idx_x]++;
        y[idx_y]++;
    }
}
// Only ONE thread can execute loop body at a time
// Essentially sequential!
```

#### Using Atomic (faster)
```cpp
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp atomic
    x[idx_x]++;
    
    #pragma omp atomic
    y[idx_y]++;
}
// Threads updating DIFFERENT indices can execute concurrently
// Example: Thread 0 updates x[0], Thread 1 updates x[1] simultaneously
```

### Performance Comparison

| Implementation | Time (ms) | Speedup |
|----------------|-----------|--------|
| Without protection | 0.128 | N/A (incorrect) |
| Critical section | 2.451 | Slowdown! |
| Atomic | 1.245 | 1.97x faster |

**Atomic is ~2x faster than critical section** for this workload!

<a id='section8'></a>
## Summary and Key Concepts

### Synchronization Mechanisms

| Mechanism | Use Case | Cost | Scope |
|-----------|----------|------|-------|
| **Critical Section** | Any protected code | High | Multiple statements |
| **Atomic** | Simple updates | Low | Single statement |
| **Reduction** | Accumulation | Very Low | Loop variable |
| **Flush** | Memory consistency | Medium | Specific variables |

### When to Use What

#### Use Reduction
```cpp
// Sum, product, min, max operations
#pragma omp parallel for reduction(+:sum)
for (int i = 0; i < N; i++) {
    sum += array[i];
}
```

#### Use Atomic
```cpp
// Simple array updates
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp atomic
    histogram[bin]++;
}
```

#### Use Critical
```cpp
// Complex operations, I/O, multiple statements
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp critical
    {
        update_data_structure();
        log_to_file();
    }
}
```

#### Use Sections + Flush
```cpp
// Producer-consumer, pipeline parallelism
#pragma omp parallel sections
{
    #pragma omp section
    producer();
    
    #pragma omp section
    consumer();
}
```

### Common Mistakes to Avoid

#### 1. Forgetting synchronization
```cpp
// BAD
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    counter++;  // Race condition!
}
```

#### 2. Too much synchronization
```cpp
// BAD: Critical section in loop body
#pragma omp parallel for
for (int i = 0; i < N; i++) {
    #pragma omp critical  // Executes N times!
    sum += array[i];
}
// GOOD: Use reduction instead
```

#### 3. Wrong scope of atomic
```cpp
// BAD: Atomic only protects ONE statement
#pragma omp atomic
x[idx]++;
y[idx]++;  // NOT protected!
```

#### 4. Missing flush in producer-consumer
```cpp
// BAD: Consumer may never see updated flag
data = value;
flag = true;  // Missing flush!
```

### Performance Tips

1. **Minimize critical sections**: Make them as small as possible
2. **Prefer reduction**: OpenMP optimizes it for you
3. **Use atomic** for simple updates: Hardware support makes it fast
4. **Avoid busy waiting**: Use tasks or taskwait for long waits
5. **Profile first**: Identify real bottlenecks before optimizing

### Next Steps

In Practical 5, you will learn:
- Barriers (explicit and implicit)
- nowait clause for performance
- C++ mutex and lock_guard
- Deadlock prevention
- Advanced synchronization patterns

---

**End of Practical 4**